# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 9 2023 2:31PM,257072,19,2022102851,"GUSA Granules USA, Inc.",Released
1,Feb 9 2023 2:00PM,257069,19,MSP218408,"Methapharm, Inc.",Released
2,Feb 9 2023 2:00PM,257069,19,MSP218414,"Methapharm, Inc.",Released
3,Feb 9 2023 1:49PM,257068,19,ADV80011548,"AdvaGen Pharma, Ltd",Released
4,Feb 9 2023 1:49PM,257068,19,ADV80011549,"AdvaGen Pharma, Ltd",Released
5,Feb 9 2023 1:48PM,257067,19,9137251,"Dr. Raju's Pharma Solutions, LLC",Released
6,Feb 9 2023 1:04PM,257062,10,DNMD-34097,Emerginnova,Released
7,Feb 9 2023 1:03PM,257061,10,SNAP28232,"Snap Medical Industries, LLC",Released
8,Feb 9 2023 1:03PM,257061,10,SNAP34728858,"Snap Medical Industries, LLC",Released
9,Feb 9 2023 12:55PM,257058,10,0086368772,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
36,257062,Released,1
37,257067,Released,1
38,257068,Released,2
39,257069,Released,2
40,257072,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
257062,NaN,1.0
257067,NaN,1.0
257068,NaN,2.0
257069,NaN,2.0
257072,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
256934,27.0,16.0
256950,0.0,2.0
256987,0.0,1.0
256993,1.0,2.0
256995,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
256934,27,16
256950,0,2
256987,0,1
256993,1,2
256995,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,256934,27,16
1,256950,0,2
2,256987,0,1
3,256993,1,2
4,256995,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,256934,27,16
1,256950,,2
2,256987,,1
3,256993,1,2
4,256995,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 9 2023 2:31PM,257072,19,"GUSA Granules USA, Inc."
1,Feb 9 2023 2:00PM,257069,19,"Methapharm, Inc."
3,Feb 9 2023 1:49PM,257068,19,"AdvaGen Pharma, Ltd"
5,Feb 9 2023 1:48PM,257067,19,"Dr. Raju's Pharma Solutions, LLC"
6,Feb 9 2023 1:04PM,257062,10,Emerginnova
7,Feb 9 2023 1:03PM,257061,10,"Snap Medical Industries, LLC"
9,Feb 9 2023 12:55PM,257058,10,ISDIN Corporation
13,Feb 9 2023 12:46PM,257055,12,Hush Hush
14,Feb 9 2023 12:28PM,257053,10,"Nextsource Biotechnology, LLC"
16,Feb 9 2023 12:06PM,257052,10,MedStone Pharma LLC


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Feb 9 2023 2:31PM,257072,19,"GUSA Granules USA, Inc.",,1
1,Feb 9 2023 2:00PM,257069,19,"Methapharm, Inc.",,2
2,Feb 9 2023 1:49PM,257068,19,"AdvaGen Pharma, Ltd",,2
3,Feb 9 2023 1:48PM,257067,19,"Dr. Raju's Pharma Solutions, LLC",,1
4,Feb 9 2023 1:04PM,257062,10,Emerginnova,,1
5,Feb 9 2023 1:03PM,257061,10,"Snap Medical Industries, LLC",,2
6,Feb 9 2023 12:55PM,257058,10,ISDIN Corporation,,4
7,Feb 9 2023 12:46PM,257055,12,Hush Hush,,1
8,Feb 9 2023 12:28PM,257053,10,"Nextsource Biotechnology, LLC",,2
9,Feb 9 2023 12:06PM,257052,10,MedStone Pharma LLC,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 9 2023 2:31PM,257072,19,"GUSA Granules USA, Inc.",1,
1,Feb 9 2023 2:00PM,257069,19,"Methapharm, Inc.",2,
2,Feb 9 2023 1:49PM,257068,19,"AdvaGen Pharma, Ltd",2,
3,Feb 9 2023 1:48PM,257067,19,"Dr. Raju's Pharma Solutions, LLC",1,
4,Feb 9 2023 1:04PM,257062,10,Emerginnova,1,
5,Feb 9 2023 1:03PM,257061,10,"Snap Medical Industries, LLC",2,
6,Feb 9 2023 12:55PM,257058,10,ISDIN Corporation,4,
7,Feb 9 2023 12:46PM,257055,12,Hush Hush,1,
8,Feb 9 2023 12:28PM,257053,10,"Nextsource Biotechnology, LLC",2,
9,Feb 9 2023 12:06PM,257052,10,MedStone Pharma LLC,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 9 2023 2:31PM,257072,19,"GUSA Granules USA, Inc.",1,
1,Feb 9 2023 2:00PM,257069,19,"Methapharm, Inc.",2,
2,Feb 9 2023 1:49PM,257068,19,"AdvaGen Pharma, Ltd",2,
3,Feb 9 2023 1:48PM,257067,19,"Dr. Raju's Pharma Solutions, LLC",1,
4,Feb 9 2023 1:04PM,257062,10,Emerginnova,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Feb 9 2023 2:31PM,257072,19,"GUSA Granules USA, Inc.",1.0,NaN
1,Feb 9 2023 2:00PM,257069,19,"Methapharm, Inc.",2.0,NaN
2,Feb 9 2023 1:49PM,257068,19,"AdvaGen Pharma, Ltd",2.0,NaN
3,Feb 9 2023 1:48PM,257067,19,"Dr. Raju's Pharma Solutions, LLC",1.0,NaN
4,Feb 9 2023 1:04PM,257062,10,Emerginnova,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 9 2023 2:31PM,257072,19,"GUSA Granules USA, Inc.",1.0,0.0
1,Feb 9 2023 2:00PM,257069,19,"Methapharm, Inc.",2.0,0.0
2,Feb 9 2023 1:49PM,257068,19,"AdvaGen Pharma, Ltd",2.0,0.0
3,Feb 9 2023 1:48PM,257067,19,"Dr. Raju's Pharma Solutions, LLC",1.0,0.0
4,Feb 9 2023 1:04PM,257062,10,Emerginnova,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,4112575,70.0,82.0
12,771078,1.0,2.0
15,1027917,26.0,27.0
19,1799301,46.0,12.0
20,257027,1.0,0.0
21,256987,1.0,0.0
22,1028066,4.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,4112575,70.0,82.0
1,12,771078,1.0,2.0
2,15,1027917,26.0,27.0
3,19,1799301,46.0,12.0
4,20,257027,1.0,0.0
5,21,256987,1.0,0.0
6,22,1028066,4.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,70.0,82.0
1,12,1.0,2.0
2,15,26.0,27.0
3,19,46.0,12.0
4,20,1.0,0.0
5,21,1.0,0.0
6,22,4.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,70.0
1,12,Released,1.0
2,15,Released,26.0
3,19,Released,46.0
4,20,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,20,21,22
Status,,,,,,,
Executing,82.0,2.0,27.0,12.0,0.0,0.0,0.0
Released,70.0,1.0,26.0,46.0,1.0,1.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,20,21,22
0,Executing,82.0,2.0,27.0,12.0,0.0,0.0,0.0
1,Released,70.0,1.0,26.0,46.0,1.0,1.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,20,21,22
0,Executing,82.0,2.0,27.0,12.0,0.0,0.0,0.0
1,Released,70.0,1.0,26.0,46.0,1.0,1.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()